In [1]:
!pip install galearn

This is a hyperparameter optimization package I created as part of homework last semester.
It's sklearn-API compatible and thus also works with lightgbm.

The training data is preprocessed minimally for show. 
exchange with your own.

I'd be glad to hear your feedback and thoughts. 
Let me know how it works for you and where I can improve it.

Documentation is just in the form of docstrings at the moment but I'm working on creating a read the docs (sorry!!!).

It's fairly straightforward and fast.

In [1]:
import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import gc
from galearn import *
from joblib import Parallel, delayed
from sklearn.model_selection import KFold

from sklearn.metrics import make_scorer
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score
import lightgbm as lgb
import matplotlib.pyplot as plt 


path_submissions = '/'

target_name = 'target'
scores_folds = {}

# Note the score is negative RMSPE because cross-validate expects a utility function, don't get rid of the minus sign below!

In [1]:
# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return -np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

In [1]:
test = pd.read_parquet("../input/train-and-test-data/test.pq")
train = pd.read_parquet("../input/train-and-test-data/train.pq")

# Parameter Grid
Input the parameter grid like in GridSearch as an iterable (lists, array). I will soon make it work with scipy distributions as well.

In [1]:
params = dict()
params['boosting_type'] = ['gbdt', 'dart']
params['n_jobs'] = [-1]
params['num_leaves'] = np.arange(50,300)
params['learning_rate'] = np.linspace(0.0001, 0.5, 1000)
params['colsample_bytree'] = np.linspace(0.5, 1, 1000)
params['subsample'] = np.linspace(0.25, 0.9, 1000)
params['n_estimators'] = np.arange(70, 300)
params['min_child_weight'] = np.linspace(0.00001, 50, 1000)
params['min_child_samples'] = np.arange(3, 50)
params['reg_alpha'] = np.linspace(0.001, 1, 1000)
params['reg_lambda'] = np.linspace(0.0001, 0.5, 1000)
params['random_state'] = [42]

In [1]:
scorer = make_scorer(rmspe)

In [1]:
features = [col for col in train.columns if col not in {"time_id", "target", "row_id"}]
y = train['target']
x = train[features]

# Parameters:

scorer: sklearn scorer function, greater is better than

iterations: the number of iterations to run the

model: an sklearn API compatible model

cv: number of cross validation folds, also accepts input kfold, groupkfold objects

selection: selection algorithm to be used, see more -> galearn.selection

If you have trouble it's worth trying to tweak the probabilities

p_cross: probability of crossover (mixing params between best scoring models)

p_mutate: probability of mutation (generating new params of best scoring models)

sim_ann: Simulated Annealing, decay p_mutate and p_cross with time by rate decay

restrict_gene_pool: restrict gene_pool size by rate = decay (narrow search of parameters)

decay: used as exponential decay for probabilities and gene_pool_window

pop_size: size of population, i.e. the number of models to consider per iteration

elitism: the fixed number of individuals to make it into each iteration (keep the best models)

In [1]:
best = simulate(params, scorer, 10,
                        lgb.LGBMRegressor,
                        x, y,
                        selection = 'tournament',
                        p_cross = 0.8,
                        p_mutate = 0.5,
                        sim_ann = True)

In [1]:
print(best)

# Have fun, may the markets be with you!